In [1]:
from requests_html import HTMLSession
from bs4 import BeautifulSoup
import requests
import re
import numpy as np
import pandas as pd
import socket
from urllib3.connection import HTTPConnection
import matplotlib.pyplot as plt
import seaborn as sns
from lxml import html
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
import time

In [19]:
HTTPConnection.default_socket_options = (
    HTTPConnection.default_socket_options + [
        (socket.SOL_SOCKET, socket.SO_KEEPALIVE, 1),
        (socket.SOL_TCP, socket.TCP_KEEPIDLE, 45),
        (socket.SOL_TCP, socket.TCP_KEEPINTVL, 10),
        (socket.SOL_TCP, socket.TCP_KEEPCNT, 6)
    ] )

In [51]:
tipomoneda_vivienda=[]
precio_vivienda=[]
direccion_vivienda=[]
metros_cuadrados_vivienda=[]
años_vivienda=[]
ambientes_vivienda=[]

link_inmobiliaria=[]



for i in range(0,159):
    
    if i==0:
        link = 'https://www.argenprop.com/departamento-alquiler-localidad-capital-federal'
        
    else:
        link= 'https://www.argenprop.com/departamento-alquiler-localidad-capital-federal'+str("-pagina-{}".format(i))
        


    driver = webdriver.Chrome()



    driver.get(link)

    time.sleep(15)

    html = driver.page_source

    driver.implicitly_wait(20)

    soup= BeautifulSoup(html,'html.parser')

    seleccionador_precio =soup.find_all('p',class_='card__price')

    tipo_moneda=[]
    precio=[]
    direccion=[]
    for currency in seleccionador_precio:
        try:
            moneda=currency.find('span',class_='card__currency').text
            tipo_moneda.append(moneda)
        except:
            continue
        valor=currency.text
        precio.append(valor)


    img_l = soup.find_all('div',class_='card__agent')

    link=[]

    for i in img_l:
        if i.img== None:
            link.append('no hay')
        else:

            img_ll=i.img
            link.append(img_ll)

    link_src=[]
    for src in link:
        if src== 'no hay':
            link_src.append('No aparece Inmobiliaria')
        else:
        
            link_src.append(src['src'])
    
    seleccionador_ubicacion = soup.find_all('div',class_='card__monetary-values')

    for i in seleccionador_ubicacion:
        try:
            lugar=i.find('h2').text
            direccion.append(lugar)
        except:
            direccion.append('falta lugar')

    metros_cuadrados =[]
    años=[]
    ambientes=[]
    precisiones = soup.find_all('ul',class_='card__main-features')

    for precision in precisiones:
        try:
            sup_cub = precision.find('li').span.text
        
            metros_cuadrados.append(sup_cub)
        except:
            metros_cuadrados.append('Faltante')
        try:
            dormitorios =precision.select('li')[1].span.text
            ambientes.append(dormitorios)
        except:
            años.append('Faltante')
        try:
            tiempo = precision.select('li')[2].span.text
            años.append(tiempo)
        except:
            años.append('Faltante')

    
    tipomoneda_vivienda.append(tipo_moneda)
    precio_vivienda.append(precio)
    direccion_vivienda.append(direccion)
    metros_cuadrados_vivienda.append(metros_cuadrados)
    años_vivienda.append(años)
    ambientes_vivienda.append(ambientes)
    link_inmobiliaria.append(link_src)
    
    driver.quit()

    
    

In [53]:
df =pd.DataFrame({'tipo_moneda':tipomoneda_vivienda,'precio_vivienda':precio_vivienda,
                'direccion_vivienda':direccion_vivienda,'metros_cuadrados':metros_cuadrados_vivienda,
                'años_vivenda':años_vivienda,'ambientes':ambientes_vivienda,'inmobiliaria':link_inmobiliaria})

In [56]:
df.to_csv('argenprop.csv',index=False)

In [46]:
tipomoneda_vivienda

lista_filtrada = [x for x in tipomoneda_vivienda if x.startswith("$") or x.startswith("£")]



In [48]:
lista_filtrada

lista_separada = [x.split("$") + x.split("£") for x in lista_filtrada]

In [50]:
lista_separada

[['',
  'USD',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '$USD$$$$$$$$$$$$$$$$$$'],
 ['',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  'USD',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  'USD',
  '$$$$$$$$$USD$$$$$$$$USD'],
 ['',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  'USD',
  'USDUSDUSD',
  '',
  '',
  '$$$$$$$$$$$$$USD$USDUSDUSD$$'],
 ['',
  '',
  'USDUSDUSDUSDUSD',
  'USD',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  'USD',
  '$$USDUSDUSDUSDUSD$USD$$$$$$$$USD'],
 ['',
  '',
  '',
  'USD',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '$$$USD$$$$$$$$$$$$$$$'],
 ['',
  '',
  'USD',
  '',
  '',
  'USD',
  '',
  '',
  '',
  '',
  '',
  '',
  'USD',
  '',
  '',
  '',
  '$$USD$$$USD$$$$$$$USD$$$'],
 ['',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  'USD',
  '',
  '',
  '',
  '',
  'USDUSD',
  '',
  'USD',
  '',
  '$$$$$$$$USD$$$$$USDUSD$$U

In [37]:
link = 'https://www.argenprop.com/departamento-alquiler-localidad-capital-federal'

driver = webdriver.Chrome()

driver.get(link)

html = driver.page_source

soup= BeautifulSoup(html,'html.parser')


img_l = soup.find_all('div',class_='card__agent')

link=[]

for i in img_l:
    if i.img== None:
        link.append('no hay')
    else:

        img_ll=i.img
        link.append(img_ll)

link_src=[]
for src in link:
    if src== 'no hay':
        link_src.append('No aparece Inmobiliaria')
    else:
        
        link_src.append(src['src'])







driver.quit()





In [38]:
link_src

['https://static1.sosiva451.com/452241_a/5b0a0314-42e8-4499-8207-dc57ddca0b02_small.jpg',
 'https://static1.sosiva451.com/452241_a/5b0a0314-42e8-4499-8207-dc57ddca0b02_small.jpg',
 'https://static1.sosiva451.com/452241_a/5b0a0314-42e8-4499-8207-dc57ddca0b02_small.jpg',
 'https://static1.sosiva451.com/424262_a/2765fe5e-2c3e-45dc-8a30-ed56d54be61b_small.jpg',
 'https://static1.sosiva451.com/424262_a/2765fe5e-2c3e-45dc-8a30-ed56d54be61b_small.jpg',
 'https://static1.sosiva451.com/338472_a/58448b26-f53c-41cb-9814-a49f8097f68b_small.jpg',
 'No aparece Inmobiliaria',
 'https://static1.sosiva451.com/932101_a/cd8eb25d-6ec2-460e-a352-0d7570b54b08_small.jpg',
 'https://static1.sosiva451.com/509111_a/ee2cd05a-f48c-4fa5-84ae-5ccd1b840e81_small.jpg',
 'No aparece Inmobiliaria',
 'No aparece Inmobiliaria',
 'No aparece Inmobiliaria',
 'No aparece Inmobiliaria',
 'https://static1.sosiva451.com/509111_a/ee2cd05a-f48c-4fa5-84ae-5ccd1b840e81_small.jpg',
 'https://static1.sosiva451.com/128921_a/b8a25630-

In [35]:
link=[]

for i in img_l:
    if i.img== None:
        link.append('no hay')
    else:

        img_ll=i.img
        link.append(img_ll)

link_src=[]
for src in link:
    if src== 'no hay':
        link_src.append('No aparece Inmobiliaria')
    else:
        
        link_src.append(src['src'])
        
       

In [36]:
link_src

['https://static1.sosiva451.com/452241_a/5b0a0314-42e8-4499-8207-dc57ddca0b02_small.jpg',
 'https://static1.sosiva451.com/452241_a/5b0a0314-42e8-4499-8207-dc57ddca0b02_small.jpg',
 'https://static1.sosiva451.com/452241_a/5b0a0314-42e8-4499-8207-dc57ddca0b02_small.jpg',
 'https://static1.sosiva451.com/424262_a/2765fe5e-2c3e-45dc-8a30-ed56d54be61b_small.jpg',
 'https://static1.sosiva451.com/424262_a/2765fe5e-2c3e-45dc-8a30-ed56d54be61b_small.jpg',
 'https://static1.sosiva451.com/338472_a/58448b26-f53c-41cb-9814-a49f8097f68b_small.jpg',
 'No aparece Inmobiliaria',
 'https://static1.sosiva451.com/932101_a/cd8eb25d-6ec2-460e-a352-0d7570b54b08_small.jpg',
 'https://static1.sosiva451.com/509111_a/ee2cd05a-f48c-4fa5-84ae-5ccd1b840e81_small.jpg',
 'No aparece Inmobiliaria',
 'No aparece Inmobiliaria',
 'No aparece Inmobiliaria',
 'No aparece Inmobiliaria',
 'https://static1.sosiva451.com/509111_a/ee2cd05a-f48c-4fa5-84ae-5ccd1b840e81_small.jpg',
 'https://static1.sosiva451.com/128921_a/b8a25630-

In [14]:
direccion_vivienda

[['\n                    JUAN MARIA GUTIERREZ 3900\n                ',
  '\n                    Cavia 3000\n                ',
  '\n                    Juan Maria Gutierrez 3900\n                ',
  '\n                    Rep De Honduras 4100, Piso PB\n                ',
  '\n                    AV. HONDURAS 3700, Piso 3\n                ',
  '\n                    Av Cordoba 3300, Piso 5\n                ',
  '\n                    RAVINGIANI 1600\n                ',
  '\n                    Laprida 1800, Piso PB\n                ',
  '\n                    Gral Juan Jose Viamonte 2800, Piso 1\n                ',
  '\n                    Eva Peron 1500, Piso 3\n                ',
  '\n                    Jose A Capdevila 3000, Piso 6\n                ',
  '\n                    Ugarteche 3200, Piso 4\n                ',
  '\n                    Congreso 2300, Piso 13\n                ',
  '\n                    Av Dr H Pueyrredon 1100, Piso 6\n                ',
  '\n                

100,116,130,155

In [11]:
len(precio)

9

In [ ]:
driver.title

'Departamentos en Alquiler en Capital Federal - Página 157 - Argenprop'